In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob

#### 初始化常量

In [2]:
sources_path = "../CoreML/flower_photos/"
split_rate = 0.2
categories = glob.glob(sources_path + "*")
category_names = [x.split("/")[-1] for x in categories]
categories_count = len(category_names)
print "category_names:%s"%category_names
log_path = "logs/"
checkpoint_path = "logs/model.ckpt"

category_names:['roses', 'sunflowers', 'daisy', 'dandelion', 'tulips', 'LICENSE.txt']


#### 本地文件读取数据

In [3]:
def readData():
    train_image_paths = []
    test_image_paths = []
    for path in categories:
        image_paths = glob.glob(path + "/*.jpg")
        split_count = int(len(image_paths) * (1 - split_rate))
        train_image_paths += image_paths[:split_count]
        test_image_paths += image_paths[split_count:]

    train_labels = [x.split("/")[-2] for x in train_image_paths]
    test_labels = [x.split("/")[-2] for x in test_image_paths]
    train_label_list = [category_names.index(x) for x in train_labels]
    test_labels_list = [category_names.index(x) for x in test_labels]

    train_image_paths = np.asarray(train_image_paths)
    train_labels = np.asarray(train_labels)
    test_image_paths = np.asarray(test_image_paths)
    test_labels = np.asarray(test_labels)

    return (train_image_paths, train_label_list), (test_image_paths, test_labels_list)

#### 图片预览

In [4]:
def view_images(images,labels,count):
    with tf.Session() as sess:
        i=0
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord = coord)
        try:
            while not coord.should_stop() and i<1:
                for j in np.arange(count):
                    plt.imshow(images[j,:,:,:])
                    index = tf.argmax(labels[j])                  
                    plt.xlabel(category_names[sess.run(index)])
                    plt.show()
                i+=1
        except tf.errors.OutOfRangeError:
            print('done!')
        finally:
            coord.request_stop()
        coord.join(threads)

### 生成器，用于生产训练数据
- Returns:训练需要的数据

In [5]:
def parse_data(image,label):
    
    image_contents = tf.read_file(image)
    image = tf.image.decode_jpeg(image_contents, channels=3)
    image = tf.image.resize_image_with_crop_or_pad(image,32,32)
    image = tf.image.per_image_standardization(image)
    label = tf.one_hot(label,depth=categories_count)
    label = tf.cast(label,dtype=tf.uint8)
    
    return image,label

In [6]:
def get_batch(image, label, image_W, image_H, batch_size):
    
    image_tf = tf.cast(image, tf.string)
    label_tf = tf.cast(label, tf.int32)
    dataset = tf.data.Dataset.from_tensor_slices((image_tf,label_tf))
    dataset = dataset.shuffle(buffer_size=len(label))
    dataset = dataset.map(parse_data)
    dataset = dataset.batch(batch_size)
    iterator =  dataset.make_initializable_iterator()
    return iterator

In [7]:
keep_prob = tf.placeholder(tf.float32)
x = tf.placeholder("float",shape=[None,32,32,3])
y = tf.placeholder("float",shape=[None,categories_count])

batch_size = 50;
learning_rate = 0.003

#### 模型定义

In [8]:
def inference(images, batch_size, n_classes):
    
    ## 第一层卷积 
    w_conv1 = tf.Variable(tf.truncated_normal([3,3,3,16],stddev=0.1))
    b_conv1 = tf.Variable(tf.constant(0.1,shape=[16]))
    y_conv1 = tf.nn.conv2d(images,w_conv1,strides=[1,1,1,1], padding="SAME") + b_conv1
    h_conv1 = tf.nn.relu(y_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,3,3,1],strides=[1,2,2,1], padding="SAME")
    
    ## 第二层卷积
    w_conv2 = tf.Variable(tf.truncated_normal([3,3,16,128],stddev=0.1))
    b_conv2 = tf.Variable(tf.constant(0.1,shape=[128]))
    y_conv2 = tf.nn.conv2d(h_pool1,w_conv2,strides=[1,1,1,1], padding="SAME") + b_conv2
    h_conv2= tf.nn.relu(y_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,3,3,1],strides=[1,1,1,1], padding="SAME")
    h_pool2_flat = tf.reshape(h_pool2,[batch_size,-1])
    
    ## 全连接层    
    w_fc1 = tf.Variable(tf.truncated_normal([16*16*128,128],stddev=0.005))
    b_fc1 = tf.Variable(tf.constant(0.1,shape=[128]))
    y_fc1 = tf.matmul(h_pool2_flat, w_fc1) + b_fc1
    h_fc1 = tf.nn.relu(y_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    
    ## 全连接层
    w_fc2 = tf.Variable(tf.truncated_normal([128,128],stddev=0.005))
    b_fc2 = tf.Variable(tf.constant(0.005,shape=[128]))
    y_fc2 = tf.matmul(h_fc1_drop, w_fc2) + b_fc2
    h_fc2= tf.nn.relu(y_fc2)
    h_fc2_drop = tf.nn.dropout(h_fc1,keep_prob)
    
    ## 全连接输出层
    w_fc3 = tf.Variable(tf.truncated_normal([128,n_classes],stddev=0.005))
    b_fc3 = tf.Variable(tf.constant(0.005,shape=[n_classes]))
    y_fc3 = tf.matmul(h_fc2_drop, w_fc3) + b_fc3
    
    return y_fc3


def losses(logits, labels):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2 \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name + '/loss', loss)
    return loss


def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


def evaluation(logits, labels):
    with tf.variable_scope('accuracy') as scope:
        labels = tf.map_fn(lambda x:tf.argmax(x) ,labels)
        labels = tf.cast(label,dtype=tf.int32)
        print labels
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name + '/accuracy', accuracy)
    return accuracy

In [9]:
(train, label), (test, testLabel) = readData()
with tf.Session() as sess:
    
    iterator = get_batch(train,label,32,32,batch_size)
    train_logits = inference(x, batch_size, categories_count)
    train_loss = losses(train_logits, y)
    train_op = trainning(train_loss, learning_rate)
    train_acc = evaluation(train_logits, y)
    
    summary_op = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_path, sess.graph)
    saver = tf.train.Saver()
    
    sess.run(tf.global_variables_initializer())

    try:
        for step in range(2000):
            batch = iterator.get_next() 
            _, tra_loss, tra_acc = sess.run([train_op, train_loss, train_acc], feed_dict = {x:batch[0], y:batch[1], keep_prob:0.5})
            if step % 50 == 0:
                print('Step %d,train loss = %.2f,train occuracy = %.2f%%' % (step, tra_loss, tra_acc))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)

            if step % 2000 == 0 or (step + 1) == MAX_STEP:
                saver.save(sess, checkpoint_path, global_step=step)
            
    except tf.errors.OutOfRangeError:
        print('Done training epoch limit reached')
    finally:
        print('Done!')
        
            
            

Tensor("accuracy/Cast/x:0", shape=(2934,), dtype=int32)


ValueError: Dimensions must be equal, but are 50 and 2934 for 'accuracy/in_top_k/InTopKV2' (op: 'InTopKV2') with input shapes: [50,6], [2934], [].